# OCPS Control Notebook

This notebook demonstrates how to execute a pipeline using the OCPS.


## One-time preliminaries

Start by importing packages.  Caution: `lsst.ts.salobj` can take 25 seconds to import.

In [ ]:
import json
import subprocess
import lsst.ts.salobj as salobj

Make sure that an OpenSplice daemon has been started.  (It's OK to run this multiple times.)

`setup.env` must have been sourced in `~/notebooks/.user_setups` first.

In [ ]:
subprocess.run(["ospl", "start"])

Start a domain and an OCPS remote within it.  Select the instrument of interest (LATISS, LSSTComCam, or LSSTCam).

In [ ]:
domain = salobj.Domain()
ocps = salobj.Remote(domain, "OCPS")

instrument = "LATISS"
if instrument == "LATISS":
    instrument_info = dict(oods_name="ATArchiver", detectors=1)
elif instrument == "LSSTComCam":
    instrument_info = dict(oods_name="CCArchiver", detectors=9)
elif instrument == "LSSTCam":
    instrument_info = dict(oods_name="MTArchiver", detectors=201)
else:
    raise RuntimeError(f"Unknown instrument {instrument}")

Optionally start an Archiver/OODS remote.

In [ ]:
oods = salobj.Remote(domain, instrument_info["oods_name"])

If necessary, start the CSC and configure it to handle LATISS data.  If already started, skip this cell.

In [ ]:
ack = await ocps.cmd_start.set_start(settingsToApply=instrument)
if ack.ack != salobj.SalRetCode.CMD_COMPLETE:
    ack.print_vars()

If necessary, enable the CSC.  If already enabled, skip this cell.

In [ ]:
ack = await ocps.cmd_enable.set_start()
if ack.ack != salobj.SalRetCode.CMD_COMPLETE:
    ack.print_vars()

## Per-image preparation

Set up the data ID.  This information could also be taken from an OODS event.

In [ ]:
day_obs = 20210510
seq_num = 12

It may be desirable to wait for OODS events saying that the image has been ingested before calling `execute`.  This optional code waits for all detectors; an `execute` command could also be issued as each detector is ingested.

In [ ]:
obs_id = f"_{day_obs}{seqnum:05}"
detectors = instrument_info["detectors"]
while detectors > 0:
    ack = await oods.evt_imageInOODS.next(flush=False)
    if ack.obsid.endswith(obs_id):
        detectors -= 1

## Pipeline execution

Execute a pipeline and ensure the command was accepted.  This code runs on all detectors, but additional data query conditions could be added to run on individual detectors.

In [ ]:
ack = await ocps.cmd_execute.set_start(
    wait_done=False,
    pipeline="$OBS_LSST_DIR/pipelines/DRP.yaml#isr", version="",
    config="-c isr:doBias=False -c isr:doDark=False -c isr:doFlat=False"
           " -c isr:doFringe=False -c isr:doLinearize=False -c isr:doDefect=False",
    data_query=f"instrument='{instrument}' AND"
               f" exposure.day_obs={day_obs} AND exposure.seq_num={seq_num}"
)
if ack.ack != salobj.SalRetCode.CMD_ACK:
    ack.print_vars()

Wait for the in-progress acknowledgement with the job identifier.  This can be executed immediately after the previous cell.

In [ ]:
ack = await ocps.cmd_execute.next_ackcmd(ack, wait_done=False)
ack.print_vars()
job_id = json.loads(ack.result)["job_id"]

Wait for the command completion acknowledgement.  This can be executed anytime after the `execute` command.

In [ ]:
ack = await ocps.cmd_execute.next_ackcmd(ack)
if ack.ack != salobj.SalRetCode.CMD_COMPLETE:
    ack.print_vars()

Wait for the job result message that matches the job id we're interested in, ignoring any others (from other remotes).  This obviously needs to follow the first acknowledgement (that returns the job id) but might as well wait for the second.

In [ ]:
while True:
    msg = await ocps.evt_job_result.next(flush=False)
    response = json.loads(msg.result)
    if response["jobId"] == job_id:
        break
response

---

## Cleanup

If needed, disable the CSC, set it to standby, and close the domain.

In [ ]:
ack = await ocps.cmd_disable.set_start()
if ack.ack != salobj.SalRetCode.CMD_COMPLETE:
    ack.print_vars()

In [ ]:
ack = await ocps.cmd_standby.set_start()
if ack.ack != salobj.SalRetCode.CMD_COMPLETE:
    ack.print_vars()

In [ ]:
domain.close()